#### 제1유형

In [62]:
# 1
import pandas as pd

df = pd.read_csv("drinks.csv")
# df.info()
# df.head()

# 대륙별 맥주 소비량의 평균 - groupby()을 구함
mean_val = df.groupby('continent')['beer_servings'].mean()
print(mean_val)

# 평균이 가장 큰 대륙
cond = df['continent'] == "Europe"
df = df[cond]

# 정렬
# df_new = df.sort_values('beer_servings', ascending=False)
# print(df_new.iloc[4, 1])

# nlargest
# df.nlargest(5, 'beer_servings')
num = df.nlargest(5, 'beer_servings').iloc[4, 1]
print(num)

continent
Africa            61.471698
Asia              37.045455
Europe           193.777778
North America    145.434783
Oceania           89.687500
South America    175.083333
Name: beer_servings, dtype: float64
313


In [56]:
# 2
import pandas as pd

df = pd.read_csv("tourist.csv")
df

# 관광객 비율이 두 번째로 높은 국가
df['방문객 합계'] = df['관광'] + df['공무'] + df['사업'] + df['기타']
df['관광객 비율'] = df['관광'] / df['방문객 합계']
df

# nlargest() - '관광객 비율'이 두번째로 높은 '사업' 방문객 수
df.nlargest(2, '관광객 비율')
a = df.nlargest(2, '관광객 비율').iloc[1, 3]
print(a)

# '관광'이 두번째로 높은 국가의 '공무' 방문객 수
df.nlargest(2, '관광')
b = df.nlargest(2, '관광').iloc[1, 2]
print(b)
print(a + b)

203
238
441


In [80]:
# 3번
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("chem.csv")
# df

# 스케일링
scaler = MinMaxScaler()
df['co'] = scaler.fit_transform(df[['co']])
df['nmhc'] = scaler.fit_transform(df[['nmhc']])
df

# 표준편차
co = df['co'].std()
print(co)
nmhc = df['nmhc'].std()
print(nmhc)
print(round(co - nmhc, 3))

0.2856516497116944
0.3030617020578397
-0.017


#### 유형2

In [134]:
# 예측 칼럼(정답, 종속변수) - TotalCharges
# 평가 지표 - MAE

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

train = pd.read_csv("churn_train.csv")
test = pd.read_csv("churn_test.csv")

# 탐색
# print(train.info())
# print(train.head())
print(train.shape, test.shape)

# 칼럼 삭제
train = train.drop(columns=['customerID'], axis=1)
test = test.drop(columns=['customerID'], axis=1)
print(train.shape, test.shape)

# 변수 선택
target = train.pop('TotalCharges')

# 원-핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)
print(train.shape, test.shape)

# 레이블 인코딩
# cols = train.select_dtypes(include="object").columns
# print(cols)

# le = LabelEncoder()
# for col in cols:
#     train[col] = le.fit_transform(train[col])
#     test[col] = le.fit_transform(test[col])
# print(train.info())

# 검증 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(train, 
            target, test_size=0.2, random_state=100)
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 모델 생성 및 학습
# model = RandomForestRegressor(max_depth=3, n_estimators=150, random_state=100)
model = RandomForestRegressor(random_state=100)
model.fit(X_train, y_train)
# 검증 테스트로 예측
pred = model.predict(X_test)
# print(pred)

# 평가 지표 - MAE(실제값, 예측값)
mae = mean_absolute_error(y_test, pred)
print("MAE:", mae)

# test 자료 측정
pred2 = model.predict(test)
print(pred2)

submit = pd.DataFrame({"pred": pred2})
submit.to_csv("result.csv", index=False)

print(pd.read_csv("result.csv"))

(4116, 19) (1764, 18)
(4116, 18) (1764, 17)
(4116, 43) (1764, 43)
MAE: 881.3176636075473
[3462.1811  943.9943 4244.1104 ... 4035.5088 1233.5765 3757.6732]
           pred
0     3462.1811
1      943.9943
2     4244.1104
3      781.2308
4     1306.9133
...         ...
1759   297.5367
1760  3990.0382
1761  4035.5088
1762  1233.5765
1763  3757.6732

[1764 rows x 1 columns]


#### 유형3

In [173]:
# 1번
import pandas as pd
from statsmodels.formula.api import logit
import numpy as np

df = pd.read_csv("churn.csv")

# df.info()
# df.head()

# 로지스틱 모델 생성 및 학습
formula = """
Churn ~ AccountWeeks + ContractRenewal + DataPlan + DataUsage + CustServCalls +
DayMins + DayCalls + MonthlyCharge + OverageFee + RoamMins
"""

model = logit(formula, data=df).fit()
# print(model.summary())

# sum() -> True(1)의 합 
print(model.pvalues[:])
# Intercept 제외 
print(sum(model.pvalues[1:] > 0.05))

# 2번
print(model.pvalues < 0.05)

# 변수 선택 - DataUsage, DayMins
model2 = logit("Churn ~ DataUsage + DayMins", df).fit()
print(model2.summary())
# 회귀 계수(상수항 포함)의 합계
sum_val = sum(model2.params)
print(round(sum_val, 3))

# 3번
# 회귀 계수
coef = model2.params['DataUsage']
print(coef)

# 오즈비 계산
odds_ratio = np.exp(coef * 5)
print(round(odds_ratio, 3))

Optimization terminated successfully.
         Current function value: 0.393603
         Iterations 6
Intercept          0.015322
AccountWeeks       0.265370
ContractRenewal    0.621969
DataPlan           0.145825
DataUsage          0.019118
CustServCalls      0.063182
DayMins            0.036410
DayCalls           0.598956
MonthlyCharge      0.436350
OverageFee         0.724826
RoamMins           0.770181
dtype: float64
8
Intercept           True
AccountWeeks       False
ContractRenewal    False
DataPlan           False
DataUsage           True
CustServCalls      False
DayMins             True
DayCalls           False
MonthlyCharge      False
OverageFee         False
RoamMins           False
dtype: bool
Optimization terminated successfully.
         Current function value: 0.397599
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  Churn   No. Observations:                 1000
Model:                    

In [201]:
# 문제 2 - 다중 선형 회귀
import pandas as pd
from statsmodels.formula.api import ols

df = pd.read_csv("piq.csv")
# df

formula = "PIQ ~ Brain + Height + Weight"
model = ols(formula, df).fit()
print(model.summary())

# p-value가 가장 작은 변수
coef = model.params['Brain']
print(round(coef, 3))

# 결정계수
print(round(model.rsquared, 2))

# 새 데이터 프레임 생성
new_df = pd.DataFrame({
    "Brain": [90],
    "Height": [70],
    "Weight": [150]
})
new_df

# PIQ 예측
pred = model.predict(new_df)
print(round(pred[0]))

                            OLS Regression Results                            
Dep. Variable:                    PIQ   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     9.024
Date:                Tue, 19 Nov 2024   Prob (F-statistic):           8.25e-05
Time:                        15:42:46   Log-Likelihood:                -217.27
No. Observations:                  50   AIC:                             442.5
Df Residuals:                      46   BIC:                             450.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     74.5114     50.564      1.474      0.1